### This specifically focuses on the application of adjacency matrices and edge indices

#### We will use molecules as the application example

In [9]:
import torch

In [4]:
from rdkit import Chem
suppl = Chem.SDMolSupplier('/home/alessandro/workshop/Learning/CompBio/Data/5ht3ligs.sdf')
mol = suppl[0]

In [15]:
def bond_index(mol):
        edges = []
        for bond in iter(mol.GetBonds()):
            edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
            edges.append([bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()])
        bond_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        return edges,bond_index
    
def self_bonds(mol):
        self_edges = [[i,i] for i in range(mol.GetNumAtoms())]
        return self_edges

In [16]:
E, BI = bond_index(mol)

In [17]:
mol.GetBonds()

### Edge representation

In [20]:
E

[[0, 1],
 [1, 0],
 [0, 2],
 [2, 0],
 [0, 3],
 [3, 0],
 [1, 4],
 [4, 1],
 [1, 5],
 [5, 1],
 [2, 6],
 [6, 2],
 [2, 7],
 [7, 2],
 [3, 8],
 [8, 3],
 [5, 9],
 [9, 5],
 [5, 10],
 [10, 5],
 [7, 11],
 [11, 7],
 [9, 12],
 [12, 9],
 [12, 13],
 [13, 12],
 [13, 14],
 [14, 13],
 [13, 15],
 [15, 13],
 [14, 16],
 [16, 14],
 [14, 17],
 [17, 14],
 [15, 18],
 [18, 15],
 [16, 19],
 [19, 16],
 [4, 6],
 [6, 4],
 [8, 11],
 [11, 8],
 [18, 19],
 [19, 18]]

## Coordinate Format (COO) 

In [21]:
BI

tensor([[ 0,  1,  0,  2,  0,  3,  1,  4,  1,  5,  2,  6,  2,  7,  3,  8,  5,  9,
          5, 10,  7, 11,  9, 12, 12, 13, 13, 14, 13, 15, 14, 16, 14, 17, 15, 18,
         16, 19,  4,  6,  8, 11, 18, 19],
        [ 1,  0,  2,  0,  3,  0,  4,  1,  5,  1,  6,  2,  7,  2,  8,  3,  9,  5,
         10,  5, 11,  7, 12,  9, 13, 12, 14, 13, 15, 13, 16, 14, 17, 14, 18, 15,
         19, 16,  6,  4, 11,  8, 19, 18]])

### Converting COO into a sparse matrix with 1s at all the edges

In [23]:
coo_tensor = torch.sparse_coo_tensor(BI, torch.ones(BI.shape[1]))

In [47]:
coo_tensor.device

device(type='cpu')

### Converting back to COO format

##### returns an uncoalesced tensor therefore need to coalesce first

In [31]:
BI2 = coo_tensor.coalesce().indices()


###### has the nice side effect of sorting the data

In [42]:
BI2

tensor([[ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  4,  4,  5,  5,  5,  6,  6,
          7,  7,  8,  8,  9,  9, 10, 11, 11, 12, 12, 13, 13, 13, 14, 14, 14, 15,
         15, 16, 16, 17, 18, 18, 19, 19],
        [ 1,  2,  3,  0,  4,  5,  0,  6,  7,  0,  8,  1,  6,  1,  9, 10,  2,  4,
          2, 11,  3, 11,  5, 12,  5,  7,  8,  9, 13, 12, 14, 15, 13, 16, 17, 13,
         18, 14, 19, 14, 15, 19, 16, 18]])

In [46]:
BI2[0]

tensor([ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  4,  4,  5,  5,  5,  6,  6,
         7,  7,  8,  8,  9,  9, 10, 11, 11, 12, 12, 13, 13, 13, 14, 14, 14, 15,
        15, 16, 16, 17, 18, 18, 19, 19])

### Converting back to Edge Index/Adjacency list

In [40]:
E2 = [list(e) for e in zip(BI2.tolist()[0], BI2.tolist()[1])]

In [41]:
E2

[[0, 1],
 [0, 2],
 [0, 3],
 [1, 0],
 [1, 4],
 [1, 5],
 [2, 0],
 [2, 6],
 [2, 7],
 [3, 0],
 [3, 8],
 [4, 1],
 [4, 6],
 [5, 1],
 [5, 9],
 [5, 10],
 [6, 2],
 [6, 4],
 [7, 2],
 [7, 11],
 [8, 3],
 [8, 11],
 [9, 5],
 [9, 12],
 [10, 5],
 [11, 7],
 [11, 8],
 [12, 9],
 [12, 13],
 [13, 12],
 [13, 14],
 [13, 15],
 [14, 13],
 [14, 16],
 [14, 17],
 [15, 13],
 [15, 18],
 [16, 14],
 [16, 19],
 [17, 14],
 [18, 15],
 [18, 19],
 [19, 16],
 [19, 18]]

In [51]:
torch.Tensor(3,3, dtype=torch.float32)

TypeError: new() received an invalid combination of arguments - got (int, int, dtype=torch.dtype), but expected one of:
 * (*, torch.device device)
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
      didn't match because some of the keywords were incorrect: dtype
 * (object data, *, torch.device device)
      didn't match because some of the keywords were incorrect: dtype
